# Load Dependencies

In [2]:
from google.cloud import speech_v1
# from google.cloud.speech_v1 import enums
from pydub import AudioSegment # 간단한 오디오 편집 패키지
import numpy as np
import io

# Speech to Text (STT)

In [7]:
# STT를 활용한 욕설 필터링 프로그램 만들기
# 기본적으로 google cloud STT API는 클라이언트 - 서버 구조로 되어있다.
# 우리가 구글 API 서버로 요청을 보내면 답을 받는 형식
# 우리가 오디오 데이터를 함수를 통해 요청할 때, 오디오 데이터만 보내지는 것이 아니라 config 또한 함께 보내짐.

def sample_recognize(local_file_path):
    
    # instantiates a client
    client = speech_v1.SpeechClient() # 실제 클라이언트 만듬. 이 클라이언트 함수로 서버에 요청을 보냄

    language_code = "ko-KR"
    sample_rate_hertz = 44100

    encoding = speech_v1.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED
    
    config = { # STT는 기본적으로 오디어 데이터를 받아 text 형식으로 변환하기 때문에 오디오 데이터에 대한 기본적인 설명 필요
        # (config의 arg 값을 설정)
        "language_code": language_code, # 실제로 입력 받는 오디오 파일의 언어 (Document를 통해 각 언어에 대한 key값 찾을 수 있음)
        "sample_rate_hertz": sample_rate_hertz,
        # sample rate: 오디오 데이터를 저장할 때 1초에 몇 개의 샘플로 쪼개서 저장하는 지 의미
        "encoding": encoding, # 실제 받는 오디오파일의 파일 타입
        "enable_word_time_offsets": True, # 단어 별로 해당 단어가 나온 시간 보여줌
        "use_enhanced": True, # 좀 더 향상된 STT model 사용할 수 있음
    }
    
    with io.open(local_file_path, "rb") as f:
        content = f.read()
    
    audio = {"content": content}

    response = client.recognize(config=config, audio=audio) # 오디오 클래스와 정의한 config를 쿨라이언트에게 보냄
    
    timeline, swear_timeline, words = [], [], []
    # swear_timeline: 욕 한 타임라인, words: 나온 단어들
    for result in response.results:
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))
        
        for word in alternative.words:
            timeline.append([
                int(word.start_time.seconds * 1000 + word.start_time.seconds * (10**-6)),
                int(word.end_time.seconds * 1000 + word.end_time.seconds * (10**-6))
            ]) # 단어의 시작 시간과 끝 시간 저장
            
            words.append(word.word) # 모든 단어 저장

            if '씨발' in word.word:
                swear_timeline.append([
                    int(word.start_time.seconds * 1000 + word.start_time.seconds * (10**-6)),
                    int(word.end_time.seconds * 1000 + word.end_time.seconds * (10**-6))
                ]) # 욕이 있으면 해당 욕의 타임라인 저장
                
    return timeline, swear_timeline, words

# execute
timeline, swear_timeline, words = sample_recognize('sound/sound.mp3')

print('timeline:', timeline)
print('words:', words)
print('swear_timeline:', swear_timeline) # --> 이 부분에서 삐 소리를 넣기 위해 따로 욕 타임라인 저장

Transcript: 일어나이 씨발놈아 개새끼야 야이 개 같은 년아 씨발년아 씨발년아
timeline: [[0, 1000], [1000, 1000], [1000, 2000], [2000, 3000], [3000, 3000], [3000, 4000], [4000, 7000], [7000, 7000], [7000, 7000], [7000, 9000], [9000, 11000]]
words: ['일어나', '이', '씨발놈아', '개새끼야', '야', '이', '개', '같은', '년아', '씨발년아', '씨발년아']
swear_timeline: [[1000, 2000], [7000, 9000], [9000, 11000]]


In [8]:
# '일어나'를 0에서 1초 사이에 말 함
# '이' 라는 단어를 1에서 1.1초 사이에 말 함

# Load Original Audio File

In [4]:
sound = AudioSegment.from_file('sound/sound.mp3', format='mp3')

print(len(sound))
sound # 12.4초 짜리 오디오 파일이라는 뜻

12422


# Create Beep Sound

In [5]:
def create_beep(duration):
    sps = 44100
    freq_hz = 1000.0
    vol = 0.5

    esm = np.arange(duration / 1000 * sps)
    wf = np.sin(2 * np.pi * esm * freq_hz / sps)
    wf_quiet = wf * vol
    wf_int = np.int16(wf_quiet * 32767)

    beep = AudioSegment(
        wf_int.tobytes(), 
        frame_rate=sps,
        sample_width=wf_int.dtype.itemsize, 
        channels=1
    )

    return beep

beep = create_beep(duration=1000)
beep

# Overlay Partially

In [6]:
# 첫번째 욕설 부분에 삐 소리 넣어보기
i = 0
mixed = sound.overlay(beep, position=swear_timeline[i][0], gain_during_overlay=-20)
# AudioSegment.overlay(): 오디오A 위에 다른 오디오B를 덧입힌다.
# position: 어디에 넣을지 설정
# gain_during_overlay: 오리지널 사운드를 얼마나 줄일 것인지 설정
mixed

# Result

In [7]:
# 욕설 전체에 삐 소리 넣기
mixed_final = sound

for i in range(len(swear_timeline)):
    beep = create_beep(duration=swear_timeline[i][1] - swear_timeline[i][0])
    mixed_final = mixed_final.overlay(beep, position=swear_timeline[i][0], gain_during_overlay=-20)
    
mixed_final

# Export

In [47]:
mixed_final.export('sound/result.mp3', format='mp3')
# AudioSegment.export(): 오디오 세그멘트를 파일로 내보내기

<_io.BufferedRandom name='sound/result.mp3'>